# SQL Project
## Bhejane Online Trading Store

© Explore Data Science Academy

### Disclaimer

Both the database and instructions for this project were provided through Explore AI Academy as part of their Data Science curriculum. This challenge was completed independently in an exam format. All code solutions preceded with an "Action" header are my own work, other cells were provided by the academy as a guide.

<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem. 

Its time to flex your Ninja SQL skills! 
 
**Your mission, should you choose to accept it:**

You will receive a denormalised database consisting of two tables. Your task is to transform the database such that is in third normal form (3NF). To ensure that a consistent normalisation process is followed by each student, we have attached an Entity Relationship Diagram (ERD) which is a guideline on what tables need to be produced. Once you have normalised the database, you will be required to complete an MCQ test found under the 'Predict' tab of Athena.  

Though the normalisation process, you will be guided into acknowledging client (Bhenjane Trading) use-cases for the database, and dealing with data anomalies in SQL. 

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
Please use the below command to install sql_magic, this is the package that will assist you with SQL syntax hightlighting.
* pip install sql_magic

Remember to start each new cell with " %%read_sql "

In [1]:
# DO NOT EDIT THIS CELL
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [2]:
# DO NOT EDIT THIS CELL
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [3]:
#DO NOT EDIT THIS CELL
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [4]:
%%read_sql
--DO NOT EDIT THIS CELL
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 09:06:35 PM South Africa Standard Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

Although we have not yet covered python in the sprint at EXPLORE - you have additional resources at your disposal, one of which happens to be a data engineering expert, and she has created a brief script for you which can be used to extract the data from the csv files, and load it into a sqlite database. 

This will be included in your submission, so that the end-to-end processing of the data is visible, and repeatable for any additional consultants who may be brought on board at a later stage. 


In [5]:
# DO NOT EDIT CELL
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Explore the Denormalized Tables
Familiarize yourself with the data given in the two tables by writing your own SQL queries to explore properties of the dataset.i.e Look for data inconsistencies, anormalies, redundancies etc to guide your normalization process. 

This section will not be graded - think of it as your own scratch pad.



In [6]:
%%read_sql
-- #Use this section to explore the database by writing your own queries
SELECT *
FROM Products
LIMIT 10;

Query started at 09:06:35 PM South Africa Standard Time; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,1 x Hikvision 1080P Bullet camera,399.0
1,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,Manual,399.0
2,,,,10325354918,467.0,ZEE,Fashion / Accessories / Scarves,Grey,,ZEE 3-in-1 Unisex Gaiter,,0.0,Limited (6 months),,,139.0
3,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Makeup / Face / Found...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
4,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Shop By Brand / Estee...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
5,,,,27131187035,275.0,Estee Lauder,Beauty / Makeup / Face / Foundation,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
6,,,,39800085139,247.0,Energizer,Cameras / Cameras & Lenses / Camera Accessorie...,,South Africa,Energizer 3V CR2450 Lithium Coin Battery,,0.0,Limited (12 months),,,49.0
7,,,,43859499182,386.0,Fellowes,Office & Stationery / Stationery / Office Supp...,,South Africa,Fellowes HFC Free Air Duster - 350ml,,0.0,Limited (6 months),,1 x 350ml Free Air Duster,149.0
8,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0
9,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0


In [7]:
%%read_sql
-- #Use this section to explore the database by writing your own queries
SELECT  ProductDescription
FROM Products
LIMIT 10

Query started at 09:06:36 PM South Africa Standard Time; Query executed in 0.00 m

,ProductDescription
0,Hikvision 1080P 2MP Turbo HD IR Bullet Camera
1,Hikvision 1080P 2MP Turbo HD IR Bullet Camera
2,ZEE 3-in-1 Unisex Gaiter
3,Estee Lauder Double Wear Stay In Place Makeup
4,Estee Lauder Double Wear Stay In Place Makeup
5,Estee Lauder Double Wear Stay In Place Makeup
6,Energizer 3V CR2450 Lithium Coin Battery
7,Fellowes HFC Free Air Duster - 350ml
8,Wahl Home Pro Basic Corded 8 Piece Haircutting...
9,Wahl Home Pro Basic Corded 8 Piece Haircutting...


In [8]:
%%read_sql

SELECT * 
FROM Transactions
LIMIT 50

Query started at 09:06:36 PM South Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,Total,UserName,InvoiceDate
0,102,300507946,1523.0,DIMPHO,2020-07-02 0:00:00
1,1,43859499182,149.0,Hendrik,2020-08-05 0:00:00
2,2,614143543746,99.0,Faristha,2020-07-29 0:00:00
3,179,617566827837,3464.0,Zanele,2020-04-04 0:00:00
4,136,619659097318,3301.0,Junaid,2020-08-04 0:00:00
5,3,619659141059,75.0,JOHAN,2020-05-20 0:00:00
6,111,619659141080,550.0,Michael,2020-04-18 0:00:00
7,112,619659161354,357.0,Nico,2020-05-09 0:00:00
8,4,638142992763,44.0,Sandra,2020-03-25 0:00:00
9,5,638142992763,44.0,Sheila,2020-08-03 0:00:00


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below below target ERD create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


It is suggested that you create all your tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form
Use the above ERD sketch to create the required tables. Please label your tables as they appear in the ERD sketch

In [9]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 09:06:36 PM South Africa Standard Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [10]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height 
FROM 
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;
    

Query started at 09:06:37 PM South Africa Standard Time; Query executed in 0.00 m

In [11]:
%%read_sql

SELECT *
FROM Transactions_1NF
LIMIT 20;

Query started at 09:06:37 PM South Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,UserName,InvoiceDate,Total
0,102,300507946,DIMPHO,2020-07-02 0:00:00,1523.0
1,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
2,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
3,179,617566827837,Zanele,2020-04-04 0:00:00,3464.0
4,136,619659097318,Junaid,2020-08-04 0:00:00,3301.0
5,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
6,111,619659141080,Michael,2020-04-18 0:00:00,550.0
7,112,619659161354,Nico,2020-05-09 0:00:00,357.0
8,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
9,5,638142992763,Sheila,2020-08-03 0:00:00,44.0


In [12]:
%%read_sql

SELECT *
FROM Products_1NF
LIMIT 20;

Query started at 09:06:37 PM South Africa Standard Time; Query executed in 0.00 m

,Barcode,NavigationPath,ItemDescription,Colour,ProductDescription,Brand,Price,Quantity,PackType,Warranty,StockCountry,Weight_kg,Volume_litre,Length,Width,Height
0,300507946,Computers & Tablets / Smart Home & Connected L...,1 x Hikvision 1080P Bullet camera,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,Limited (6 months),,,0.0,,,
1,300507946,Computers & Tablets / Smart Home & Connected L...,Manual,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,Limited (6 months),,,0.0,,,
2,10325354918,Fashion / Accessories / Scarves,,Grey,ZEE 3-in-1 Unisex Gaiter,ZEE,139.0,467,,Limited (6 months),,,0.0,,,
3,27131187035,Beauty / Luxury Beauty / Makeup / Face / Found...,,Fresco,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,,Non-Returnable,South Africa,,0.0,,,
4,27131187035,Beauty / Luxury Beauty / Shop By Brand / Estee...,,Fresco,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,,Non-Returnable,South Africa,,0.0,,,
5,27131187035,Beauty / Makeup / Face / Foundation,,Fresco,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,,Non-Returnable,South Africa,,0.0,,,
6,39800085139,Cameras / Cameras & Lenses / Camera Accessorie...,,,Energizer 3V CR2450 Lithium Coin Battery,Energizer,49.0,247,,Limited (12 months),South Africa,,0.0,,,
7,43859499182,Office & Stationery / Stationery / Office Supp...,1 x 350ml Free Air Duster,,Fellowes HFC Free Air Duster - 350ml,Fellowes,149.0,386,,Limited (6 months),South Africa,,0.0,,,
8,43917915548,Health / Personal Care / Shaving & Hair Remova...,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,Wahl Home Pro Basic Corded 8 Piece Haircutting...,WAHL,149.0,338,,Limited (6 months),South Africa,,0.0,,,
9,43917915548,Health / Personal Care / Shaving & Hair Remova...,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,Wahl Home Pro Basic Corded 8 Piece Haircutting...,WAHL,149.0,338,,Limited (6 months),South Africa,,0.0,,,


###  7. Convert the database into its 2nd Normal Form (2NF).

Please label your tables as they appear in the ERD sketch.

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example, 

Thereby, we remove the partial dependancy, and will be in 2NF. We are instructed to create a unique identifier for the `Transactions_2NF`. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach. 

#### 7.3 Data Anomalies

You may stumble into an error when you insert the data which violates this constraint. You query this with the data owner, and you are told that there are three transactions in the database which were errors. You are informed that *all transactions in the database correspond to products are in the database*.

These entries were captured incorrectly, and must be removed. How will you deal with this? 

In [13]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 09:06:38 PM South Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,2


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [14]:
%%read_sql
delete from Transactions_1NF 
WHERE barcode not in (select barcode from products_1NF)

Query started at 09:06:38 PM South Africa Standard Time; Query executed in 0.00 m

In [15]:
%%read_sql
-- Should show no entries!
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 09:06:38 PM South Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that you understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint yourself!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 

You will need to work around this by removing the erroneous data from the tables before continuing, since this would violate the FK constraint between the Transactions_2NF and Products_2NF table otherwise. 

_Hint: Identify the observations in the Transactions_1NF table, and remove them from there._

#### 7.5 Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database. 

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. You are tasked with removing 'useless' entries as you take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

_Hint: Make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [16]:
%%read_sql 

SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products
WHERE ItemDescription = '';

Query started at 09:06:38 PM South Africa Standard Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


### Let's do it! Construct the database above in second normal form

In [17]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 09:06:41 PM South Africa Standard Time; Query executed in 0.00 m

### Let's do it! _Populate_ the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries. 

In [18]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Products_1NF;
    
INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;
    
INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT 
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 09:06:42 PM South Africa Standard Time; Query executed in 0.01 m

In [19]:
%%read_sql

SELECT * 
FROM Colours_2NF
LIMIT 6;

Query started at 09:06:42 PM South Africa Standard Time; Query executed in 0.00 m

,ColourID,Colour
0,1,Grey
1,2,Fresco
2,3,Red
3,4,White
4,5,Black
5,6,Transparent


A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [20]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg, 
    Volume_litre, 
    Length,
    Width,
    Height 
FROM 
    Products_1NF
LEFT JOIN Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
LEFT JOIN PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 09:06:43 PM South Africa Standard Time; Query executed in 0.00 m

In [21]:
%%read_sql

SELECT *
FROM Products_2NF

Query started at 09:06:43 PM South Africa Standard Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Brand,Price,Quantity,StockCountry,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,,0.0,,,,11,1.0,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,,0.0,,,,11,2.0,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,ZEE,139.0,467,,,0.0,,,,1,NaN,1.0
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,,0.0,,,,8,NaN,2.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,South Africa,,0.0,,,,9,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,1210,TAL00035388021,Angelcare - Nappy Bin Refill - 3 Pack,Angelcare,269.0,220,South Africa,,0.0,,,,396,599.0,7.0
1210,1211,TAL00035388021,Angelcare - Nappy Bin Refill - 3 Pack,Angelcare,269.0,220,South Africa,,0.0,,,,395,599.0,7.0
1211,1212,TAL00035388407,Pampers Complete Clean Baby Wipes - 6 x 64 - 3...,Pampers,159.0,130,South Africa,,0.0,"19,8","21,3","3,2",15,600.0,4.0
1212,1213,TAL00035394505,RCT 650VA Line Interactive UPS,RCT,675.0,410,South Africa,,"10,72678",,,,271,39.0,NaN


Last but not least, we insert into the `Transactions_2NF`table. Recall that we created a new PK in this table, and pay attention to how we are inserting values into the table. 

In [22]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 09:06:43 PM South Africa Standard Time; Query executed in 0.01 m

<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

Please label your tables as they appear in the ERD sketch. In the same fashion as done in 2NF, note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [23]:
%%read_sql

--#Create tables required for 2NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES Navigation_3NF ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES PackageContents_3NF ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES Colours_3NF ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES Brands_3NF ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES Locations_3NF ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 09:06:46 PM South Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the construction of the remaining tables mentioned </font>

In [24]:
%%read_sql

CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserID" INTEGER NOT NULL,
    "CartID" INTEGER NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName"),
    CONSTRAINT fk_trans FOREIGN KEY ("Barcode") REFERENCES Products_3NF ("Barcode"),
    CONSTRAINT fk_trans FOREIGN KEY ("UserID") REFERENCES Users_3NF ("UserID"),
    CONSTRAINT fk_trans FOREIGN KEY ("CartID") REFERENCES Carts_3NF ("CartID")
);
    
                            
CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150) NOT NULL,
    PRIMARY KEY("PathID" AUTOINCREMENT)
);


CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150) NOT NULL,
    "PackType" VARCHAR(150) NOT NULL,
    "Warranty" VARCHAR(150) NOT NULL,
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);


CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR(150),
    PRIMARY KEY("BrandID" AUTOINCREMENT)
);

    
CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    PRIMARY KEY("LocationID" AUTOINCREMENT)
);



Query started at 09:06:47 PM South Africa Standard Time; Query executed in 0.00 m

In [25]:
%%read_sql

--# Populate the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName FROM Transactions_2NF;


Query started at 09:06:49 PM South Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Carts_3NF`.   </font>

Take note that there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [26]:
%%read_sql

INSERT INTO "Carts_3NF"("InvoiceDate","Total")
SELECT DISTINCT 
    InvoiceDate,
    Total
FROM Transactions_2NF;
    


Query started at 09:06:49 PM South Africa Standard Time; Query executed in 0.00 m

In [27]:
%%read_sql

SELECT COUNT(*)
FROM Carts_3NF


Query started at 09:06:49 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT(*)
0,190


In [28]:
%%read_sql

SELECT COUNT(*)
FROM Users_3NF
LIMIT 5;

Query started at 09:06:49 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT(*)
0,158


### <font color='turquoise'>Action: proceed with the insertion into the tables below. </font>

Similarly to the table above, mind the occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows. 


In [29]:
%%read_sql

INSERT INTO "Locations_3NF" ("StockCountry")
SELECT DISTINCT
    StockCountry
FROM Products_2NF
WHERE StockCountry <> '';

INSERT INTO "Brands_3NF" ("Brand")
SELECT DISTINCT
    Brand
FROM Products_2NF
WHERE Brand <> '';
    

INSERT INTO "Colours_3NF" ("Colour")
SELECT DISTINCT
    Colour 
FROM Colours_2NF;

INSERT INTO "PackageContents_3NF" ("ItemDescription","PackType","Warranty")
SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty
FROM PackageContents_2NF;



INSERT INTO "Navigation_3NF" ("NavigationPath")
SELECT DISTINCT
    NavigationPath
FROM Navigation_2NF;




Query started at 09:06:50 PM South Africa Standard Time; Query executed in 0.00 m

In [30]:
%%read_sql

SELECT *
FROM Brands_3NF
LIMIT 10;

Query started at 09:06:51 PM South Africa Standard Time; Query executed in 0.00 m

,BrandID,Brand
0,1,Hikvision
1,2,ZEE
2,3,Estee Lauder
3,4,Energizer
4,5,Fellowes
5,6,WAHL
6,7,Bombay Sapphire
7,8,Digitronics
8,9,Intelli-Vision Technology
9,10,SanDisk


In [31]:
%%read_sql

SELECT *
FROM Colours_2NF
LIMIT 3;

Query started at 09:06:51 PM South Africa Standard Time; Query executed in 0.00 m

,ColourID,Colour
0,1,Grey
1,2,Fresco
2,3,Red


In [32]:
%%read_sql

SELECT *
FROM Colours_3NF
LIMIT 3;

Query started at 09:06:51 PM South Africa Standard Time; Query executed in 0.00 m

,ColourID,Colour
0,1,Grey
1,2,Fresco
2,3,Red


### <font color='turquoise'>Action: proceed with the insertion into `Products_3NF`. </font>
Take note of the joins required for additional data contained in other tables. 

In [33]:
%%read_sql

INSERT INTO "Products_3NF" ("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT DISTINCT
    PathID,
    ItemID,
    ColourID,
    BrandID,
    LocationID,
    Products_2NF.Barcode, 
    ProductDescription,
    Price,
    Quantity,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height
FROM
    Products_2NF
LEFT JOIN Brands_3NF
    ON Products_2NF.Brand = Brands_3NF.Brand
LEFT JOIN Locations_3NF
    ON Products_2NF.StockCountry = Locations_3NF.StockCountry;


Query started at 09:06:51 PM South Africa Standard Time; Query executed in 0.01 m

In [34]:
%%read_sql

SELECT COUNT (*)
FROM Locations_3NF
LIMIT 5

Query started at 09:06:51 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT (*)
0,2


In [35]:
%%read_sql

SELECT *
FROM Products_3NF
LIMIT 5;


Query started at 09:06:52 PM South Africa Standard Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,1,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,1.0,NaN,1,NaN
1,2,300507946,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,399.0,493,,0.0,,,,11,2.0,NaN,1,NaN
2,3,10325354918,ZEE 3-in-1 Unisex Gaiter,139.0,467,,0.0,,,,1,NaN,1.0,2,NaN
3,4,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,8,NaN,2.0,3,1.0
4,5,27131187035,Estee Lauder Double Wear Stay In Place Makeup,655.0,275,,0.0,,,,9,NaN,2.0,3,1.0


### <font color='turquoise'>Action: proceed with the insertion into `Transactions_3NF`.   </font>

Follow previous proceedure, and checks. 

In [36]:
%%read_sql

INSERT INTO "Transactions_3NF" ("CartID||Barcode||UserName", "CartID","Barcode","UserID")
SELECT DISTINCT
    [CartID||Barcode||UserName],
    CartID,
    Barcode,
    UserID
FROM Transactions_2NF
LEFT JOIN Users_3NF
    ON Users_3NF.UserName = Transactions_2NF.UserName;


Query started at 09:06:52 PM South Africa Standard Time; Query executed in 0.01 m

In [37]:
%%read_sql

SELECT *
FROM Transactions_3NF
LIMIT 5;

Query started at 09:06:52 PM South Africa Standard Time; Query executed in 0.00 m

,CartID||Barcode||UserName,Barcode,UserID,CartID
0,143859499182Hendrik,43859499182,1,1
1,2614143543746Faristha,614143543746,2,2
2,3619659141059JOHAN,619659141059,3,3
3,4638142992763Sandra,638142992763,4,4
4,5638142992763Sheila,638142992763,5,5


### <font color='turquoise'>Action: Investigate the DB.   </font>

Note - the tables counts below are what you should arrive at. If you don't, reconsider how you have done the inserts above. Having the tables correctly formatted will allow to answer the questions more easily, and without errors.

| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [38]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
select 'Products_3NF' as table_name, count(*) from Products_3NF
UNION
select 'Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select 'Users_3NF' as table_name, count(*) from Users_3NF
UNION
select 'Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select 'PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select 'Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select 'Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select 'Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select 'Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 09:06:52 PM South Africa Standard Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,396
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


<a id='MCQ_questions'></a>
## 9. MCQ Questions

Having completed the normalisation of the database, you may use the following cells to help you answer the MCQs for the predict. 

Q1) How many unique products does the company have?

In [39]:
%%read_sql

SELECT COUNT(DISTINCT Barcode)
FROM Products_3NF;


Query started at 09:06:52 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT Barcode)
0,608


Q2) How many users bought from Bhejane in April 2020?

In [40]:
%%read_sql

SELECT COUNT(DISTINCT t.UserID) AS "no. of users"
FROM Carts_3NF c
JOIN Transactions_3NF t
ON t.CartID = c.CartID
WHERE InvoiceDate BETWEEN "2020-04-01 0:00:00" AND "2020-04-30 0:00:00"


Query started at 09:06:53 PM South Africa Standard Time; Query executed in 0.00 m

,no. of users
0,31


In [41]:
%%read_sql

SELECT COUNT(DISTINCT t.UserID) AS "no. of users", TRIM(SUBSTR(InvoiceDate, 6,2)) AS "month"
FROM Carts_3NF c
JOIN Transactions_3NF t
ON t.CartID = c.CartID
WHERE [month] = "04"


Query started at 09:06:53 PM South Africa Standard Time; Query executed in 0.00 m

,no. of users,month
0,31,04


In [42]:
%%read_sql

SELECT * 
FROM Carts_3NF
LIMIT 1

Query started at 09:06:53 PM South Africa Standard Time; Query executed in 0.00 m

,CartID,InvoiceDate,Total
0,1,2020-08-05 0:00:00,149.0


Q3) How many users bought 3 or more items that cost more than R1000?

In [146]:
%%read_sql

SELECT t.UserID, c.Total, t.Barcode, c.CartID
FROM Transactions_3NF t
JOIN Carts_3NF c
ON t.CartID = c.CartID
WHERE c.Total > 1000


Query started at 09:55:57 PM South Africa Standard Time; Query executed in 0.00 m

,UserID,Total,Barcode,CartID
0,7,1999.0,741311301174,7
1,19,2299.0,5012583201885,22
2,48,1999.0,6005519007674,56
3,84,1334.0,6001106128616,99
4,84,1334.0,8714574605555,99
5,84,1334.0,HURACANNANOWAVE1,99
6,84,1334.0,MPTAL00233916,99
7,85,1523.0,6001865825405,100
8,85,1523.0,6001878003128,100
9,85,1523.0,MPTAL72574833,100


In [46]:
%%read_sql
-- Write your query here:

SELECT DISTINCT Barcode AS "items"
FROM Products
WHERE Price > 1000


Query started at 09:08:50 PM South Africa Standard Time; Query executed in 0.00 m

,items
0,193808791192
1,711719870258
2,741311301174
3,815592024309
4,5060563450051
5,5397063622078
6,5397184092651
7,5397184200193
8,6003000640348
9,6005519114211


In [ ]:
%%read_sql
-- Write your query here:

SELECT 




Q4) Which user made the largest purchase on a single transaction?

In [154]:
%%read_sql
-- Write your query here:

SELECT MAX(c.Total), t.UserID, u.UserName, c.CartID
FROM Carts_3NF c
JOIN Transactions_3NF t
ON c.CartID = t.CartID
JOIN Users_3NF u
ON u.UserID = t.UserID
GROUP BY u.UserName
ORDER BY c.Total DESC



Query started at 09:58:47 PM South Africa Standard Time; Query executed in 0.00 m

,MAX(c.Total),UserID,UserName,CartID
0,10999.0,102,Wilfred Nkosinathi,121
1,10999.0,103,Vincent,122
2,3765.0,64,Mandla,123
3,3765.0,83,Brigette,124
4,3464.0,147,Rolf,177
...,...,...,...,...
151,29.0,56,Casey,67
152,23.0,33,Nick,38
153,13.0,38,Siphesihle,43
154,12.0,50,Leonard,59


In [ ]:
%%read_sql
-- Write your query here:

SELECT Total, CartID, InvoiceDate
FROM Carts_3NF
ORDER BY Total DESC
LIMIT 5;



In [ ]:
%%read_sql

SELECT *
FROM Carts_3NF
LIMIT 5

Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [66]:
%%read_sql
-- Write your query here:

SELECT DISTINCT p.Barcode, p.ProductDescription AS "product", pc.ItemDescription AS "components"
FROM Products_3NF p
JOIN PackageContents_3NF pc
ON pc.ItemID = p.ItemID
WHERE p.Barcode LIKE "MPTAL57588104"


Query started at 09:19:05 PM South Africa Standard Time; Query executed in 0.00 m

,Barcode,product,components
0,MPTAL57588104,5m Colour Changing RGB LED Strip Light,1 x 5 Meter LED RGB Strip Light
1,MPTAL57588104,5m Colour Changing RGB LED Strip Light,1 x 24 key IR remote controller
2,MPTAL57588104,5m Colour Changing RGB LED Strip Light,1 x IR Controller
3,MPTAL57588104,5m Colour Changing RGB LED Strip Light,1 x 12V 3A Power Supply


In [ ]:
%%read_sql
-- Write your query here:

SELECT COUNT(DISTINCT pc.ItemDescription)
FROM Products_3NF p
JOIN PackageContents_3NF pc
ON pc.ItemID = p.ItemID
WHERE p.Barcode LIKE "MPTAL57588104"


In [ ]:
%%read_sql
-- Write your query here:

SELECT ItemDescription AS Components
FROM PackageContents_3NF
LIMIT 6


Q6) How many brands are available at Bhejane?

In [ ]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT Brand)
FROM Brands_3NF

7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [110]:
%%read_sql
-- Write your query here:

SELECT DISTINCT p.Price
FROM Products_3NF p
WHERE p.ProductDescription = "Verimark - Floorwiz 2in1 Mop"

Query started at 09:40:33 PM South Africa Standard Time; Query executed in 0.00 m

,Price
0,179.0


In [111]:
%%read_sql
-- Write your query here:

SELECT ProductDescription
FROM Products_3NF p
LIMIT 6

Query started at 09:40:39 PM South Africa Standard Time; Query executed in 0.00 m

,ProductDescription
0,Hikvision 1080P 2MP Turbo HD IR Bullet Camera
1,Hikvision 1080P 2MP Turbo HD IR Bullet Camera
2,ZEE 3-in-1 Unisex Gaiter
3,Estee Lauder Double Wear Stay In Place Makeup
4,Estee Lauder Double Wear Stay In Place Makeup
5,Estee Lauder Double Wear Stay In Place Makeup


Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [ ]:
%%read_sql
-- Write your query here:

SELECT ROUND(Width * Height * Length) AS "product_volume"
FROM Products_3NF
WHERE ProductDescription = "Russell Hobbs - Slow Cooker"

Q9) Which user made the most transactions in the Year 2020?

In [99]:
%%read_sql

SELECT UserName
FROM(
SELECT u.UserName, c.InvoiceDate
FROM Users_3NF u
JOIN Transactions_3NF t
ON u.UserID = t.UserID
JOIN Carts_3NF c
ON c.CartID = t.CartID
GROUP BY c.InvoiceDate
ORDER BY u.UserName )



Query started at 09:32:49 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT(UserName)
0,132


In [171]:
%%read_sql

SELECT t.UserID, c.Total, c.CartID
FROM Transactions_3NF t
JOIN Carts_3NF c
ON t.CartID = c.CartID
JOIN Users_3NF u
ORDER BY t.UserID


Query started at 10:04:20 PM South Africa Standard Time; Query executed in 0.00 m

,UserID,Total,CartID
0,1,149.0,1
1,1,149.0,1
2,1,149.0,1
3,1,149.0,1
4,1,149.0,1
...,...,...,...
42655,156,757.0,190
42656,156,757.0,190
42657,156,757.0,190
42658,156,757.0,190


In [182]:
%%read_sql

SELECT Total, COUNT(*) AS "woop", InvoiceDate
FROM Carts_3NF 

GROUP BY Total
Order BY [woop] 


Query started at 10:11:52 PM South Africa Standard Time; Query executed in 0.00 m

,Total,woop,InvoiceDate
0,1.0,1,2020-04-21 0:00:00
1,10.0,1,2020-05-17 0:00:00
2,12.0,1,2020-07-01 0:00:00
3,13.0,1,2020-05-08 0:00:00
4,29.0,1,2020-07-16 0:00:00
...,...,...,...
125,349.0,3,2020-08-07 0:00:00
126,249.0,4,2020-04-08 0:00:00
127,99.0,5,2020-07-29 0:00:00
128,149.0,5,2020-08-05 0:00:00


In [184]:
%%read_sql

SELECT InvoiceDate, COUNT(*)
FROM Carts_3NF
GROUP BY InvoiceDate

Query started at 10:12:46 PM South Africa Standard Time; Query executed in 0.00 m

,InvoiceDate,COUNT(*)
0,2020-03-25 0:00:00,1
1,2020-03-26 0:00:00,1
2,2020-03-27 0:00:00,2
3,2020-03-28 0:00:00,2
4,2020-03-29 0:00:00,2
...,...,...
128,2020-08-09 0:00:00,2
129,2020-08-10 0:00:00,2
130,2020-08-11 0:00:00,2
131,2020-08-12 0:00:00,2


In [192]:
%%read_sql

SELECT UserID, COUNT(*) AS "hey"
FROM Transactions_3NF
GROUP BY UserID
ORDER BY [hey] DESC

Query started at 10:16:34 PM South Africa Standard Time; Query executed in 0.00 m

,UserID,hey
0,9,9
1,31,8
2,120,6
3,6,6
4,64,5
...,...,...
153,8,1
154,4,1
155,3,1
156,2,1


In [195]:
%%read_sql

SELECT UserID, UserName
FROM Users_3NF
WHERE UserID = "9"

Query started at 10:17:21 PM South Africa Standard Time; Query executed in 0.00 m

,UserID,UserName
0,9,Cornelis


In [ ]:
%%read_sql
-- Write your query here:

SELECT u.UserName, COUNT(c.CartID) AS "number", c.InvoiceDate
FROM Users_3NF u
JOIN Transactions_3NF t
ON u.UserID = t.UserID
JOIN Carts_3NF c
ON c.CartID = t.CartID
GROUP BY c.CartID
ORDER BY [number] DESC
LIMIT 5


In [65]:
%%read_sql
-- Write your query here:

SELECT CartID, Barcode 
FROM Transactions
GROUP BY CartID


Query started at 09:18:14 PM South Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode
0,1,43859499182
1,2,614143543746
2,3,619659141059
3,4,638142992763
4,5,638142992763
...,...,...
187,188,MPTAL72781093
188,189,MPTAL72849953
189,190,MPTAL72849953
190,191,MPTAL72849955


Q10) What is the total number of users that shop at Bhejane?

In [ ]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT UserID)
FROM Transactions_3NF

Q11) What is the record count for the Colours_3NF Table?

In [ ]:
%%read_sql
-- Write your query here:

SELECT COUNT(DISTINCT Colour)
FROM Colours_3NF

In [197]:
%%read_sql
-- Write your query here:

SELECT COUNT(DISTINCT Colour)
FROM Colours_3NF

Query started at 10:20:14 PM South Africa Standard Time; Query executed in 0.00 m

,COUNT(Colour)
0,17


Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [ ]:
%%read_sql
-- Write your query here:
SELECT SUM(Price)
FROM (
SELECT DISTINCT Price 
FROM Products_3NF
WHERE Barcode = "MPTAL57588104"
UNION
SELECT DISTINCT Price
FROM Products_3NF
WHERE Barcode = "5000394203921"
UNION
SELECT DISTINCT Price
FROM Products_3NF
WHERE Barcode = "6932391917652"
    )


In [ ]:
%%read_sql
-- Write your query here:

SELECT DISTINCT Price 
FROM Products
WHERE Barcode = "MPTAL57588104"
UNION
SELECT DISTINCT Price
FROM Products
WHERE Barcode = "5000394203921"
UNION
SELECT DISTINCT Price
FROM Products
WHERE Barcode = "6932391917652"


Q13) What is the barcode of the most sold product?

In [ ]:
%%read_sql
-- Write your query here:
-- look for max count barcode?

SELECT t.Barcode, c.CartID
FROM Transactions_3NF t
JOIN Carts_3NF c
ON c.CartID = t.CartID
ORDER BY t.Barcode DESC




In [196]:
%%read_sql

SELECT Barcode, COUNT(*) AS "hey"
FROM Transactions_3NF
GROUP BY Barcode
ORDER BY [hey] DESC

Query started at 10:18:01 PM South Africa Standard Time; Query executed in 0.00 m

,Barcode,hey
0,4015400541813,4
1,MPTAL00404857,3
2,HURACANNANOWAVE1,3
3,6009686620537,3
4,6001865825405,3
...,...,...
216,5000394020146,1
217,4902505163104,1
218,43859499182,1
219,4210201043577,1


Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [133]:
%%read_sql
-- Write your query here:
-- invoice date join transactions join users join products

SELECT u.UserName AS "shopper", c.InvoiceDate AS "date", p.ProductDescription as "ïtems"
FROM Users_3NF u
JOIN Transactions_3NF t
ON u.UserID = t.UserID
JOIN Carts_3NF c
ON c.CartID = t.CartID
JOIN Products_3NF p
ON p.Barcode = t.Barcode
WHERE [shopper] = "Cornelis"
ORDER BY [shopper]




Query started at 09:50:35 PM South Africa Standard Time; Query executed in 0.00 m

,shopper,date,ïtems
0,Cornelis,2020-07-23 0:00:00,Pampers Premium Care - Size 5 Mega Pack - 88 N...
1,Cornelis,2020-08-09 0:00:00,KN95 Dual Breathing Face Mask
2,Cornelis,2020-08-05 0:00:00,Dettol Body Wash - Shower Gel - Original - 600ml
3,Cornelis,2020-08-05 0:00:00,Vitality Aloe Vera Gel 100ml
4,Cornelis,2020-08-05 0:00:00,Vitality Aloe Vera Gel 100ml
5,Cornelis,2020-08-05 0:00:00,Vitality Aloe Vera Gel 100ml
6,Cornelis,2020-04-10 0:00:00,Montego - Classic Adult Dog Succulent Beef & W...
7,Cornelis,2020-04-10 0:00:00,HDMI 10m Braided Cable
8,Cornelis,2020-08-05 0:00:00,Sparq Active Resistance Loop Band - Set of 5 w...
9,Cornelis,2020-08-05 0:00:00,Sparq Active Resistance Loop Band - Set of 5 w...


Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [ ]:
%%read_sql
-- Write your query here:
-- locationid == 1 (local), colourID == 5 (black), extract month using substring
-- subquery to get total cost rounded

SELECT SUM(Total)
FROM
(
    SELECT u.UserName AS "customer", p.ColourID, p.LocationID, TRIM(SUBSTR(InvoiceDate, 6,2)) AS "trimdate", c.Total
     FROM Users_3NF u
     JOIN Transactions_3NF t
     ON u.UserID = t.UserID
     JOIN Carts_3NF c
     ON c.CartID = t.CartID
     JOIN Products_3NF p
     ON p.Barcode = t.Barcode
     WHERE (LocationID = "1") AND (ColourID = "5") AND ([trimdate] = "01" OR  [trimdate] = "03" OR  [trimdate] = "05" OR  [trimdate] = "07" OR  [trimdate] = "09")
)




In [ ]:
%%read_sql
-- Write your query here:
-- locationid == 1 (local), colourID == 5 (black), extract month using substring
-- subquery to get total cost rounded

SELECT ROUND(SUM(Total))
FROM
(
    SELECT u.UserName AS "customer", p.ColourID AS "colour", p.LocationID AS "location", TRIM(SUBSTR(InvoiceDate, 6,2)) AS "trimdate", c.Total, u.UserID, c.CartID
     FROM Users_3NF u
     JOIN Transactions_3NF t
     ON u.UserID = t.UserID
     JOIN Carts_3NF c
     ON c.CartID = t.CartID
     JOIN Products_3NF p
     ON p.Barcode = t.Barcode
     WHERE (LocationID = "1") AND (ColourID = "5") AND ([trimdate] = "01" OR  [trimdate] = "03" OR  [trimdate] = "05" OR  [trimdate] = "07" OR  [trimdate] = "09")
     ORDER BY c.CartID
) 



In [ ]:
%%read_sql
-- Write your query here:
-- locationid == 1 (local), colourID == 5 (black), extract month using substring
-- subquery to get total cost rounded


    SELECT u.UserName AS "customer", p.ColourID AS "colour", p.LocationID AS "location", TRIM(SUBSTR(InvoiceDate, 6,2)) AS "trimdate", c.InvoiceDate, c.Total, u.UserID, c.CartID
     FROM Users_3NF u
     JOIN Transactions_3NF t
     ON u.UserID = t.UserID
     JOIN Carts_3NF c
     ON c.CartID = t.CartID
     JOIN Products_3NF p
     ON p.Barcode = t.Barcode
     WHERE (LocationID = "1") AND (ColourID = "5") AND ([trimdate] = "01" OR  [trimdate] = "03" OR  [trimdate] = "05" OR  [trimdate] = "07" OR  [trimdate] = "09")
     GROUP BY c.Total
     ORDER BY c.CartID 
     



In [ ]:
%%read_sql
-- Write your query here:
-- locationid == 1 (local), colourID == 5 (black), extract month using substring
-- subquery to get total cost rounded

SELECT SUM(Total)
FROM
(
    SELECT u.UserName AS "customer", p.ColourID AS "colour", p.LocationID AS "location", TRIM(SUBSTR(InvoiceDate, 6,2)) AS "trimdate", c.Total, u.UserID, c.CartID
     FROM Users_3NF u
     JOIN Transactions_3NF t
     ON u.UserID = t.UserID
     JOIN Carts_3NF c
     ON c.CartID = t.CartID
     JOIN Products_3NF p
     ON p.Barcode = t.Barcode
     WHERE (LocationID = "1") AND (ColourID = "5") AND ([trimdate] = "01" OR  [trimdate] = "03" OR  [trimdate] = "05" OR  [trimdate] = "07" OR  [trimdate] = "09")
     GROUP BY c.Total
     ORDER BY c.CartID 
)     



In [ ]:
%%read_sql

SELECT TRIM(SUBSTR(InvoiceDate, 6,2)) AS "trimdate"
FROM Carts_3NF
WHERE [trimdate] = "01" OR  [trimdate] = "03" OR  [trimdate] = "05" OR  [trimdate] = "07" OR  [trimdate] = "09"